# End-to-End Runbook

This notebook runs the full GridPulse workflow: data → features → splits → training → API smoke test.

In [ ]:
from pathlib import Path
import sys
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
print('Python:', sys.executable)
print('repo_root:', repo_root)

## 1) Install local package (editable)

In [ ]:
import subprocess
venv_python = repo_root / '.venv' / 'bin' / 'python'
python_bin = str(venv_python) if venv_python.exists() else 'python'
subprocess.run([python_bin, '-m', 'pip', 'install', '-e', str(repo_root)], check=False)

## 2) Ensure raw OPSD file exists

In [ ]:
from pathlib import Path
raw_dir = repo_root / 'data' / 'raw'
raw_csv = raw_dir / 'time_series_60min_singleindex.csv'
nested = raw_dir / 'opsd-time_series-2020-10-06' / 'time_series_60min_singleindex.csv'
if raw_csv.exists():
    print('Raw file present:', raw_csv)
elif nested.exists():
    raw_dir.mkdir(parents=True, exist_ok=True)
    raw_csv.write_bytes(nested.read_bytes())
    print('Copied raw file from nested OPSD folder')
else:
    print('Raw OPSD file not found. Download or place it at:')
    print(raw_csv)

## 3) Run data pipeline

In [ ]:
import subprocess
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.validate_schema', '--in', str(repo_root/'data'/'raw'), '--report', str(repo_root/'reports'/'data_quality_report.md')], check=False)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.build_features', '--in', str(repo_root/'data'/'raw'), '--out', str(repo_root/'data'/'processed')], check=False)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.split_time_series', '--in', str(repo_root/'data'/'processed'/'features.parquet'), '--out', str(repo_root/'data'/'processed'/'splits')], check=False)

## 4) Train forecasting models

In [ ]:
from pathlib import Path
features_path = repo_root / 'data' / 'processed' / 'features.parquet'
if not features_path.exists():
    print('features.parquet not found. Run step 3 first.')
else:
    import subprocess
    subprocess.run(['python', '-m', 'gridpulse.forecasting.train', '--config', str(repo_root/'configs'/'train_forecast.yaml')], check=False)

## 5) Start API (separate terminal)

```bash
uvicorn services.api.main:app --reload --port 8000
```

## 6) API smoke test

In [ ]:
import requests
base = 'http://localhost:8000'
try:
    print(requests.get(f'{base}/health', timeout=5).json())
except Exception as e:
    print('API not reachable. Start it in a separate terminal.', e)

In [ ]:
import requests
base = 'http://localhost:8000'
try:
    print(requests.get(f'{base}/forecast', timeout=5).json().get('meta', {}))
except Exception as e:
    print('Forecast call failed:', e)

In [ ]:
import requests
base = 'http://localhost:8000'
payload = {
    'forecast_load_mw': [8000, 8200, 8100],
    'forecast_renewables_mw': [3200, 3100, 3300]
}
try:
    print(requests.post(f'{base}/optimize', json=payload, timeout=10).json())
except Exception as e:
    print('Optimize call failed:', e)

## 7) Start the dashboard

```bash
streamlit run services/dashboard/app.py
```